**Set environment**

In [8]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /data/reddylab/Kuei 
WORK DIRECTORY:     /data/reddylab/Kuei/out 
CODE DIRECTORY:     /data/reddylab/Kuei/code 
PATH OF SOURCE:     /data/reddylab/Kuei/source 
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin 
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation 
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 


## Import data

In [9]:
ASSAY    = "A001_K562_WSTARRseq"
FOLDER   = "coverage_astarrseq_peak_macs_input"

In [10]:
fdiry = file.path(FD_RES, "results", ASSAY, FOLDER, "summary")
fname = "result.Log2FC.raw.deseq.WGS.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_log2fc = dat
print(dim(dat))
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/summary/result.Log2FC.raw.deseq.WGS.tsv"
[1] 246688      7


Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1_17237_17772,15.507847,-0.55197423,0.3650312,-1.5121288,0.1305011090,0.376522564
chr1_136071_137429,8.005396,0.42184778,0.5134014,0.8216725,0.4112633282,NA
chr1_137737_139544,68.388785,0.75905245,0.1909129,3.9759105,0.0000701105,0.001384044
chr1_180982_182087,35.093278,0.19192129,0.2417529,0.7938737,0.4272689717,0.696317984
chr1_183239_184602,49.565727,0.08374879,0.2018363,0.4149342,0.6781900855,0.857342193
chr1_186238_187159,50.450003,-0.13877450,0.2161170,-0.6421267,0.5207908905,0.763065578


In [11]:
fdiry = file.path(FD_RES, "results", "comparison")
fname = "dat_peaks_info.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

###
dat_peak_info = dat
dat_peak      = dat %>% dplyr::select(Peak, Dataset, Region)
print(dim(dat))
head(dat)

[1] 1451    6


Peak,Chrom,Start,End,Dataset,Region
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr11_61792068_61793464,chr11,61792068,61793464,OL13,FADS
chr11_61800085_61801113,chr11,61800085,61801113,OL13,FADS
chr11_61806630_61807154,chr11,61806630,61807154,OL13,FADS
chr11_61814735_61817343,chr11,61814735,61817343,OL13,FADS
chr11_61822094_61822443,chr11,61822094,61822443,OL13,FADS
chr11_61825795_61826306,chr11,61825795,61826306,OL13,FADS


## Rank normalization

In [12]:
fun_ranknorm = function(x){
    return(rank(x)/length(x))
}

In [13]:
dat = left_join(dat_peak, dat_log2fc, by="Peak")

dat = dat %>%
    dplyr::group_by(Region) %>%
    dplyr::mutate(RankNorm_Region  = fun_ranknorm(log2FoldChange)) %>%
    dplyr::ungroup()

dat = dat %>%
    dplyr::group_by(Dataset) %>%
    dplyr::mutate(RankNorm_Dataset = fun_ranknorm(log2FoldChange)) %>%
    dplyr::ungroup()

dat = dat %>%
    dplyr::mutate(RankNorm_Total  = fun_ranknorm(log2FoldChange))

dat_ranknorm = dat
print(dim(dat))
tail(dat)

[1] 1451   12


Peak,Dataset,Region,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,RankNorm_Region,RankNorm_Dataset,RankNorm_Total
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr2_61524551_61524988,OL45,BCL11A,17.94124,-0.80352554,0.3431467,-2.3416387,1.919929e-02,1.155411e-01,0.02717391,0.01062802,0.01171606
chr2_61529854_61531204,OL45,BCL11A,41.53372,0.05517938,0.2210035,0.2496765,8.028375e-01,9.199271e-01,0.73913043,0.58357488,0.56443832
chr2_61536312_61537087,OL45,BCL11A,80.19413,-0.25100377,0.1657820,-1.5140588,1.300109e-01,3.757547e-01,0.40217391,0.26763285,0.24741558
chr2_61537852_61539255,OL45,BCL11A,217.94586,1.02196259,0.1123886,9.0931155,9.624095e-20,3.609483e-17,1.00000000,0.98550725,0.98345968
chr2_61545554_61545869,OL45,BCL11A,16.46355,-0.45494185,0.3407968,-1.3349357,1.818974e-01,4.508779e-01,0.13043478,0.09661836,0.08752584
chr2_61549710_61550633,OL45,BCL11A,61.93789,-0.27116546,0.1792901,-1.5124393,1.304221e-01,3.763766e-01,0.35326087,0.24637681,0.22674018


In [14]:
fdiry = file.path(FD_RES, "results", "comparison", "comparison_global")
fname = "result.ranknorm.wstarr.tsv"
fpath = file.path(fdiry, fname)

dat = dat_ranknorm
write_tsv(dat, fpath)